<a href="https://colab.research.google.com/github/RajuDasa/llm_engineering/blob/week2_day4/week2/community-contributions/raju/week2_day4_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tools using - update flight price:**

In [ ]:
!pip install litellm

In [ ]:
from litellm import completion

In [ ]:
from google.colab import userdata
import os

## set ENV variables
#os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

In [ ]:
#DB update with initial data
import sqlite3

DB = "airlines.db"

#admin table
def add_login_table():
  with sqlite3.connect(DB) as con:
    cur = con.cursor()
    cur.execute('CREATE TABLE IF NOT EXISTS login (userId TEXT, password TEXT)')
    con.commit()
    cur.execute('INSERT INTO login (userId, password) VALUES (?, ?)', ("admin", "admin123"))
    con.commit()

#flight price table
def add_airline_table():
  initial_prices = [("london", 1250),("paris", 2200),("dubai",1550)]
  with sqlite3.connect(DB) as con:
    cur = con.cursor()
    cur.execute('CREATE TABLE IF NOT EXISTS flight_price (city TEXT PRIMARY KEY, price REAL)')
    con.commit()
    add_prices(initial_prices)

#add/update records
def add_prices(coll):
  coll = [(city.lower(), price, price) for city,price in coll]
  with sqlite3.connect(DB) as con:
    cur = con.cursor()
    cur.execute("BEGIN")
    cur.executemany('INSERT INTO flight_price (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', coll)
    con.commit()


In [ ]:
add_login_table()
add_airline_table()

In [ ]:
#format chat message
def get_message(role, msg=None, content=None):
  dic = dict()
  dic['role']=role
  if(msg):
      dic['content']=msg
      return [dic]
  elif(content):  #use with gradio history
      dic['content']=content[0]["text"]
      return dic


system_message = """
You are a concise and helpful assistant that updates flight prices in a database for one city at a time (origin: Hyderabad).
You must follow the rules below strictly:

1. Authentication Required -
Before performing any price update, you must confirm that the user is authenticated.
If the user is not authenticated, request their username and password (in one message).
Call the authentication tool once credentials are provided.
If authentication fails, inform the user briefly and request credentials again.
Once authenticated, do not ask again in this session.

2. Single-City Update Restriction -
If the user requests to update more than one city in a single message, respond with a short sentence stating you can only process one city at a time.
If exactly one city and its price are provided, call the price-update tool.

3. Tool Usage -
Always use the provided tools for authentication and updating flight prices - never fabricate results.

4. Response Style -
Keep every response to one short sentence.
"""

chat_history = get_message("system", msg=system_message)

def reset_history():
  global chat_history
  chat_history = get_message("system", msg=system_message)

In [ ]:
#helper pkg to prepare JSON schema for our tool functions

!pip install function-schema

In [ ]:
#Tools

def authenticate(username: str, password: str) -> str:
  """returns authentication success or failed status"""

  with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT 1 FROM login WHERE userId = ? AND password=?', (username, password))
    result = cursor.fetchone()
    return "authentication " + ("success" if result else "failed")

def update_price(city: str, price: float) -> str:
  """returns update operation success or failed status"""

  try:
    # with sqlite3.connect(DB) as conn:
    #   cursor = conn.cursor()
    #   cursor.execute('UPDATE flight_price SET price = ? WHERE city = ?', (price, city.lower()))
    #   conn.commit()

    add_prices([(city, price)])
    return "update success"
  except Exception as e:
    return "update failed"

func_list = {
    "authenticate" : authenticate,
    "update_price" : update_price
}

In [ ]:
from function_schema import get_function_schema

tools = []

for func in func_list.values():
  schema = get_function_schema(func)
  tools.append({"type": "function", "function": schema})

#display(tools)

In [ ]:
import json

def handle_tool_calls(message):
  responses = []
  for tool_call in message.tool_calls:
    arguments = json.loads(tool_call.function.arguments)
    tool = func_list.get(tool_call.function.name, lambda x,y: "Tool not found")
    result = tool(**arguments)
    responses.append({
        "role": "tool",
        "content": result,
        "tool_call_id": tool_call.id
    })
  return responses

In [ ]:
#Model call  (Note: not all free models support tool calling in OpenRouter)

#MODEL = "openrouter/z-ai/glm-4.5-air:free"  #limited tokens

MODEL = "gemini/gemini-2.5-flash"

def call_model(user_msg, history):
  #ignore history arg as it won't track tool role.

  global chat_history
  chat_history += get_message("user", msg=user_msg)

  response = completion(
    model = MODEL,
    messages = chat_history,
    tools = tools
  )
  if response.choices[0].finish_reason=="tool_calls":
    message = response.choices[0].message
    responses = handle_tool_calls(message)
    chat_history.append(message)
    chat_history.extend(responses)
    response = completion(model=MODEL, messages=chat_history)

  result = response.choices[0].message.content
  chat_history += get_message("assistant", msg=result)
  return result

In [ ]:
import gradio as gr

gr.ChatInterface(type="messages", fn=call_model).launch(share=False, debug=True)
#Ctrl+C to stop

Run below required cells manually

In [ ]:
#check chat history

chat_history

In [ ]:
# Refresh the history before you re-run the above gradio code
reset_history()

In [ ]:
#Test/check data in DB

with sqlite3.connect(DB) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM flight_price")
    result = cur.fetchall()
    display(result)